In [1]:
import pathlib
import re
import asyncio
import logging
import sys

from collections import deque
from elasticsearch import Elasticsearch,helpers
from elasticsearch.helpers import parallel_bulk
from elasticsearch.helpers import streaming_bulk
from es_pandas import es_pandas
from elasticsearch_dsl import Search

import ujson as json
#import commond.ipynb from same folder
import import_ipynb
from common import *

logger = logging.getLogger(__name__)
handler = logging.StreamHandler()
formatter = logging.Formatter(
    fmt='%(asctime)s [%(levelname)s] %(name)s: %(message)s',
)
handler.setFormatter(formatter)
handler.setLevel(logging.DEBUG)
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)

importing Jupyter notebook from common.ipynb
03 Aug 2020 DAQ FilesPQC887


In [2]:
def get_create_index_dsl(shrads=1):
    request_body = {'settings': {'number_of_shards': shrads,
                    'number_of_replicas': 2},
                    'mappings': {'properties': {'timestamp': {'type': 'date',
                    'format': 'yyyy-MM-dd HH:mm:ss||yyyy-MM-dd||epoch_millis'}}}}

    return request_body

In [3]:
def delete_index(es, index_name):
    if es.indices.exists(index_name):
        res = es.indices.delete(index=index_name)
        print("delete res: ", res)
    else:
        print("%s does not existing" % index_name)

In [4]:
def create_index(es, index_name, shards):
    if es.indices.exists(index_name):
        print(f"{index_name} exists")

    request_body = get_create_index_dsl(shards)
    res = es.indices.create(index=index_name, body=request_body, timeout="60s")
    print("%s is created" % index_name)
    print('%s is done!' % sys._getframe().f_code.co_name)

In [5]:
def ingrest_bulk_json(json_file, index_name):
    try:
        # make the bulk call, and get a response
        results = list(parallel_bulk(client=es, actions=json_file, index=index_name,chunk_size=1000, thread_count=4, queue_size=16))
        elf.assertTrue(len(set([r[1] for r in results])) > 1)
        print ("\nRESPONSE:", results)
    except Exception as e:
        print("\nERROR:", e)
    print('%s is done!' % sys._getframe().f_code.co_name)

In [6]:
es_host = "http://elasticsearch-7.apaas-ppe1.eniot.io"
index_name = "benchmark_base"
shards = 1

In [7]:
## Create index
es = Elasticsearch(es_host, port=80)
es.indices.create(index='new-index', request_timeout=5)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'new-index'}

In [8]:
## Delete Index

es = Elasticsearch(es_host, port=80)
dict = es.indices.delete(index='new-index')

In [9]:
## use DSL to create index
index_name = "raw_index_test"
shards = 1

delete_index(es, index_name)
es = Elasticsearch(es_host, port=80)

if create_index(es, index_name, shards):
    print('<%s> is created successfuly'% index_name)


delete res:  {'acknowledged': True}
raw_index_test is created
create_index is done!


In [10]:
# Ingest bulk from json file, which is generated from filename.
json_file = 'sample.json'

ingrest_bulk_json(json_file, index_name)


ERROR: ('11 document(s) failed to index.', [{'index': {'_index': 'raw_index_test', '_type': '_doc', '_id': 'CWPMd3kBmGua5MiEF7p8', 'status': 400, 'error': {'type': 'mapper_parsing_exception', 'reason': 'failed to parse', 'caused_by': {'type': 'not_x_content_exception', 'reason': 'Compressor detection can only be called on some xcontent bytes or compressed xcontent bytes'}}, 'data': 's'}}, {'index': {'_index': 'raw_index_test', '_type': '_doc', '_id': 'CmPMd3kBmGua5MiEF7p8', 'status': 400, 'error': {'type': 'mapper_parsing_exception', 'reason': 'failed to parse', 'caused_by': {'type': 'not_x_content_exception', 'reason': 'Compressor detection can only be called on some xcontent bytes or compressed xcontent bytes'}}, 'data': 'a'}}, {'index': {'_index': 'raw_index_test', '_type': '_doc', '_id': 'C2PMd3kBmGua5MiEF7p8', 'status': 400, 'error': {'type': 'mapper_parsing_exception', 'reason': 'failed to parse', 'caused_by': {'type': 'not_x_content_exception', 'reason': 'Compressor detection c

**list all index**
GET /_cat/indices  
